In [1]:
# Bibliotecas
import os
import dotenv
dotenv.load_dotenv()
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import inspect, create_engine
import requests
import zipfile

engine = create_engine("postgresql://"+os.getenv('DB_USERNAME')+":"+os.getenv('REMOTE_PSQL_PASSWORD')+"@"+os.getenv('REMOTE_PSQL_IP')+":"+os.getenv('DB_PORT')+"/"+os.getenv('DB_DATABASE')+"")

def convert_bytes(size):
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024.0:
            return "%3.1f %s" % (size, x)
        size /= 1024.0
    return size

pd.set_option("display.max_columns", None)

In [2]:
# FOR 2018 BULLETINS
df2018 = pd.read_csv("/media/abreufilho/HD - Abreu Filho/Shud/shud_spaces/eleicoes/2018/boletins/bweb_1t_MG_101020181954/bweb_1t_MG_101020181954.csv", encoding="latin1", sep=";",
    dtype={
        "NR_PARTIDO":str,
        "DS_AGREGADAS":str,
        "NR_JUNTA_APURADORA":str,
        "NR_TURMA_APURADORA":str,
        "CD_FLASHCARD_URNA_EFETIVADA":str,
    },
    #nrows=10,
    usecols=["ANO_ELEICAO","CD_PLEITO","DT_PLEITO","NR_TURNO","CD_ELEICAO","DS_ELEICAO","SG_ UF","CD_MUNICIPIO","NM_MUNICIPIO","NR_PARTIDO","SG_PARTIDO","NM_PARTIDO","NR_VOTAVEL","NM_VOTAVEL","NR_ZONA","NR_SECAO","NR_LOCAL_VOTACAO","QT_APTOS","QT_COMPARECIMENTO","QT_ABSTENCOES","NR_VOTAVEL","QT_VOTOS"]
)

cols_to_rename = {
    'CD_FLASCARD_URNA_EFETIVADA':'CD_FLASHCARD_URNA_EFETIVADA',
    "SG_ UF":"SG_UF" # Typo na coluna SG_UF
}
df2018 = df2018.rename(columns=cols_to_rename)

df2018 = df2018.loc[df2018["SG_UF"] == "MG"]

df2018['NR_PARTIDO'] = df2018['NR_PARTIDO'].replace('#NULO#','-1', regex=True)
df2018["NR_PARTIDO"] = pd.to_numeric(df2018["NR_PARTIDO"])
##############################################################################################################################################
# FOR 2018 BULLETINS
df2020 = pd.read_csv("/media/abreufilho/HD - Abreu Filho/Shud/shud_spaces/eleicoes/2020/resultados/boletins_urna/1_turno/bweb_1t_MG_181120201549/bweb_1t_MG_181120201549.csv", encoding="latin1", sep=";",
    dtype={
        "NR_PARTIDO":str,
        "DS_AGREGADAS":str,
        "NR_JUNTA_APURADORA":str,
        "NR_TURMA_APURADORA":str,
        "CD_FLASHCARD_URNA_EFETIVADA":str,
    },
    #nrows=10,
    usecols=["ANO_ELEICAO","CD_PLEITO","DT_PLEITO","NR_TURNO","CD_ELEICAO","DS_ELEICAO","SG_UF","CD_MUNICIPIO","NM_MUNICIPIO","NR_PARTIDO","SG_PARTIDO","NM_PARTIDO","NR_VOTAVEL","NM_VOTAVEL","NR_ZONA","NR_SECAO","NR_LOCAL_VOTACAO","QT_APTOS","QT_COMPARECIMENTO","QT_ABSTENCOES","NR_VOTAVEL","QT_VOTOS"]
)

df2020 = df2020.loc[df2020["SG_UF"] == "MG"]

df2020['NR_PARTIDO'] = df2020['NR_PARTIDO'].replace('#NULO#','-1', regex=True)
df2020["NR_PARTIDO"] = pd.to_numeric(df2020["NR_PARTIDO"])
##############################################################################################################################################
df = df2018.append(df2020)

In [3]:
df

,ANO_ELEICAO,CD_PLEITO,DT_PLEITO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS
0,2018,228,07/10/2018,1,295,Eleição Geral Federal 2018,MG,40002,ALFREDO VASCONCELOS,25,115,1015,-1,#NULO#,#NULO#,231,214,17,95,Branco,8
1,2018,228,07/10/2018,1,295,Eleição Geral Federal 2018,MG,40002,ALFREDO VASCONCELOS,25,115,1015,-1,#NULO#,#NULO#,231,214,17,96,Nulo,26
2,2018,228,07/10/2018,1,295,Eleição Geral Federal 2018,MG,40002,ALFREDO VASCONCELOS,25,115,1015,12,PDT,Partido Democrático Trabalhista,231,214,17,12,CIRO GOMES,33
3,2018,228,07/10/2018,1,295,Eleição Geral Federal 2018,MG,40002,ALFREDO VASCONCELOS,25,115,1015,13,PT,Partido dos Trabalhadores,231,214,17,13,FERNANDO HADDAD,66
4,2018,228,07/10/2018,1,295,Eleição Geral Federal 2018,MG,40002,ALFREDO VASCONCELOS,25,115,1015,15,MDB,Movimento Democrático Brasileiro,231,214,17,15,HENRIQUE MEIRELLES,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3968721,2020,304,15/11/2020,1,426,Eleições Municipais 2020,MG,45950,IBIRITÉ,351,195,1023,20,PSC,Partido Social Cristão,343,306,37,20456,ANTONIO DO OURO NEGRO,33
3968722,2020,304,15/11/2020,1,426,Eleições Municipais 2020,MG,45950,IBIRITÉ,351,195,1023,19,PODE,Podemos,343,306,37,19333,MARCOS GARAPA,2
3968723,2020,304,15/11/2020,1,426,Eleições Municipais 2020,MG,45950,IBIRITÉ,351,195,1023,36,PTC,Partido Trabalhista Cristão,343,306,37,36555,ELIAS SANTOS,1
3968724,2020,304,15/11/2020,1,426,Eleições Municipais 2020,MG,45950,IBIRITÉ,351,195,1023,11,PP,PROGRESSISTAS,343,306,37,11655,FABINHO ARAÚJO,13


In [ ]:
# ADICIONA COLUNA QT_VOTOS NA TABELA CANDIDATOS (get_votes_for_candidates.py)
columns = list(df)
sum_columns = ["QT_VOTOS"]
agg_dict = {}
for column_name in columns:
    if column_name not in sum_columns:
        agg_dict[column_name] = "first"
    else:
        agg_dict[column_name] = "sum"
df2 = df.groupby(["NM_VOTAVEL", "NR_VOTAVEL", "CD_ELEICAO", "SG_UF"], as_index=False, dropna=False).agg(agg_dict)
df2.drop(["QT_APTOS","QT_COMPARECIMENTO","QT_ABSTENCOES","NR_ZONA","NR_SECAO","NR_LOCAL_VOTACAO","CD_MUNICIPIO", "NM_MUNICIPIO"], axis=1, inplace=True)
candidatos = pd.read_sql_table('candidates', con=engine, schema="public", index_col="id")
processed_candidatos = df2.merge(candidatos, left_on=["NM_VOTAVEL", "NR_VOTAVEL", "CD_ELEICAO", "SG_UF"], right_on=["NM_URNA_CANDIDATO", "NR_CANDIDATO", "CD_ELEICAO", "SG_UF"], how='right', suffixes=('', '_df2'))
extra_columns = [item for item in list(processed_candidatos) if item not in list(candidatos)]
extra_columns.remove("QT_VOTOS")
processed_candidatos.drop(extra_columns, axis=1, inplace=True)
processed_candidatos.to_sql("candidates", con=engine, schema="public", if_exists='replace', index="id")

In [ ]:
# GERA TABELA VOTES_SCHOOLS (bulletins_to_votes_schools.py)
columns = list(df)
sum_columns = ["QT_VOTOS","QT_APTOS","QT_COMPARECIMENTO","QT_ABSTENCOES"]
agg_dict = {}
for column_name in columns:
    if column_name not in sum_columns:
        agg_dict[column_name] = "first"
    else:
        agg_dict[column_name] = "sum"
df2 = df.groupby(["NR_ZONA", "NR_LOCAL_VOTACAO", "CD_ELEICAO", "NR_VOTAVEL"], as_index=False, dropna=False).agg(agg_dict)
df2.drop(["NR_SECAO"], axis=1, inplace=True)
df2.to_sql("votes_schools", con=engine, schema="public", if_exists='replace', index="id")
df2

In [11]:
# GERA TABELA VOTES_NEIGHBORHOODS (bulletins_to_votes_neighborhood.py)
schools = pd.read_sql_table('schools', con=engine, schema="public", index_col="id")
df2 = df.merge(schools, on=['NR_ZONA', 'NR_LOCAL_VOTACAO'], how='outer', suffixes=('', '_df2'))
columns = list(df2)
sum_columns = ["QT_VOTOS","QT_APTOS","QT_COMPARECIMENTO","QT_ABSTENCOES"]
agg_dict = {}
for column_name in columns:
    if column_name not in sum_columns:
        agg_dict[column_name] = "first"
    else:
        agg_dict[column_name] = "sum"
df2 = df2.groupby(["NM_MUNICIPIO", "NM_BAIRRO", "CD_ELEICAO", "NR_VOTAVEL"], as_index=False).agg(agg_dict)
df2.drop(["SG_UF_df2", "NM_MUNICIPIO_df2", "NR_ZONA", "NR_LOCAL_VOTACAO", "NR_SECAO", "CD_MUNICIPIO_df2", "NM_LOCAL_VOTACAO", "DS_ENDERECO", "NR_CEP"], axis=1, inplace=True)
df2 = df2.loc[df2['QT_VOTOS'] > 0]
df2.to_sql("votes_neighborhoods", con=engine, schema="public", if_exists='replace', index="id")
df2

,ANO_ELEICAO,CD_PLEITO,DT_PLEITO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NM_BAIRRO
0,2018.0,228.0,07/10/2018,1.0,295.0,Eleição Geral Federal 2018,MG,40010.0,ABADIA DOS DOURADOS,12.0,PDT,Partido Democrático Trabalhista,4852.0,3910.0,942.0,12.0,CIRO GOMES,414.0,CENTRO
1,2018.0,228.0,07/10/2018,1.0,295.0,Eleição Geral Federal 2018,MG,40010.0,ABADIA DOS DOURADOS,13.0,PT,Partido dos Trabalhadores,4852.0,3910.0,942.0,13.0,FERNANDO HADDAD,1163.0,CENTRO
2,2018.0,228.0,07/10/2018,1.0,295.0,Eleição Geral Federal 2018,MG,40010.0,ABADIA DOS DOURADOS,15.0,MDB,Movimento Democrático Brasileiro,4028.0,3270.0,758.0,15.0,HENRIQUE MEIRELLES,32.0,CENTRO
3,2018.0,228.0,07/10/2018,1.0,295.0,Eleição Geral Federal 2018,MG,40010.0,ABADIA DOS DOURADOS,16.0,PSTU,Partido Socialista dos Trabalhadores Unificado,321.0,253.0,68.0,16.0,VERA,1.0,CENTRO
4,2018.0,228.0,07/10/2018,1.0,295.0,Eleição Geral Federal 2018,MG,40010.0,ABADIA DOS DOURADOS,17.0,PSL,Partido Social Liberal,4852.0,3910.0,942.0,17.0,JAIR BOLSONARO,1480.0,CENTRO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300385,2020.0,304.0,15/11/2020,1.0,426.0,Eleições Municipais 2020,MG,40193.0,ÁGUAS VERMELHAS,70.0,AVANTE,AVANTE,783.0,573.0,210.0,70865.0,FRED PINTOR,5.0,DISTRITO DE ÁGUAS ALTAS
2300386,2020.0,304.0,15/11/2020,1.0,426.0,Eleições Municipais 2020,MG,40193.0,ÁGUAS VERMELHAS,70.0,AVANTE,AVANTE,3127.0,2303.0,824.0,70888.0,JAMILLE DE JAI,85.0,DISTRITO DE ÁGUAS ALTAS
2300387,2020.0,304.0,15/11/2020,1.0,426.0,Eleições Municipais 2020,MG,40193.0,ÁGUAS VERMELHAS,70.0,AVANTE,AVANTE,3127.0,2303.0,824.0,70999.0,DUDU DO MOCÓ,20.0,DISTRITO DE ÁGUAS ALTAS
2300388,2020.0,304.0,15/11/2020,1.0,426.0,Eleições Municipais 2020,MG,40193.0,ÁGUAS VERMELHAS,77.0,SOLIDARIEDADE,Solidariedade,1959.0,1454.0,505.0,77123.0,JEAN DE TATÚ,8.0,DISTRITO DE ÁGUAS ALTAS


In [12]:
# GERA TABELA VOTES_CITIES (bulletins_to_votes_neighborhood.py)
schools = pd.read_sql_table('schools', con=engine, schema="public", index_col="id")
df2 = df.merge(schools, on=['NR_ZONA', 'NR_LOCAL_VOTACAO'], how='outer', suffixes=('', '_df2'))
columns = list(df2)
sum_columns = ["QT_VOTOS","QT_APTOS","QT_COMPARECIMENTO","QT_ABSTENCOES"]
agg_dict = {}
for column_name in columns:
    if column_name not in sum_columns:
        agg_dict[column_name] = "first"
    else:
        agg_dict[column_name] = "sum"
df2 = df2.groupby(["NM_MUNICIPIO", "CD_ELEICAO", "NR_VOTAVEL"], as_index=False).agg(agg_dict)
df2.drop(["SG_UF_df2", "NM_MUNICIPIO_df2", "NR_ZONA", "NR_LOCAL_VOTACAO", "NR_SECAO", "CD_MUNICIPIO_df2", "NM_LOCAL_VOTACAO", "DS_ENDERECO", "NR_CEP"], axis=1, inplace=True)
df2 = df2.loc[df2['QT_VOTOS'] > 0]
df2.to_sql("votes_cities", con=engine, schema="public", if_exists='replace', index="id")
df2

,ANO_ELEICAO,CD_PLEITO,DT_PLEITO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NM_BAIRRO
0,2018.0,228.0,07/10/2018,1.0,295.0,Eleição Geral Federal 2018,MG,40010.0,ABADIA DOS DOURADOS,12.0,PDT,Partido Democrático Trabalhista,5772.0,4616.0,1156.0,12.0,CIRO GOMES,476.0,CENTRO
1,2018.0,228.0,07/10/2018,1.0,295.0,Eleição Geral Federal 2018,MG,40010.0,ABADIA DOS DOURADOS,13.0,PT,Partido dos Trabalhadores,5772.0,4616.0,1156.0,13.0,FERNANDO HADDAD,1347.0,CENTRO
2,2018.0,228.0,07/10/2018,1.0,295.0,Eleição Geral Federal 2018,MG,40010.0,ABADIA DOS DOURADOS,15.0,MDB,Movimento Democrático Brasileiro,4827.0,3885.0,942.0,15.0,HENRIQUE MEIRELLES,42.0,CENTRO
3,2018.0,228.0,07/10/2018,1.0,295.0,Eleição Geral Federal 2018,MG,40010.0,ABADIA DOS DOURADOS,16.0,PSTU,Partido Socialista dos Trabalhadores Unificado,321.0,253.0,68.0,16.0,VERA,1.0,CENTRO
4,2018.0,228.0,07/10/2018,1.0,295.0,Eleição Geral Federal 2018,MG,40010.0,ABADIA DOS DOURADOS,17.0,PSL,Partido Social Liberal,5772.0,4616.0,1156.0,17.0,JAIR BOLSONARO,1732.0,CENTRO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473107,2020.0,304.0,15/11/2020,1.0,426.0,Eleições Municipais 2020,MG,40193.0,ÁGUAS VERMELHAS,77.0,SOLIDARIEDADE,Solidariedade,2349.0,1754.0,595.0,77123.0,JEAN DE TATÚ,9.0,CENTRO
473108,2020.0,304.0,15/11/2020,1.0,426.0,Eleições Municipais 2020,MG,40193.0,ÁGUAS VERMELHAS,77.0,SOLIDARIEDADE,Solidariedade,2969.0,2306.0,663.0,77333.0,DEIA DO PARA TERRA,13.0,CENTRO
473109,2020.0,304.0,15/11/2020,1.0,426.0,Eleições Municipais 2020,MG,40193.0,ÁGUAS VERMELHAS,77.0,SOLIDARIEDADE,Solidariedade,2249.0,1740.0,509.0,77444.0,ZE PAU PODRE,8.0,CENTRO
473110,2020.0,304.0,15/11/2020,1.0,426.0,Eleições Municipais 2020,MG,40193.0,ÁGUAS VERMELHAS,77.0,SOLIDARIEDADE,Solidariedade,4107.0,3188.0,919.0,77555.0,DÁDÁ DO PROGRESSO,28.0,CENTRO
